<a href="https://colab.research.google.com/github/GrigoryBartosh/hse08_ip/blob/master/hw3_classic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd 'drive/My Drive/ip_hw_3'

/content/drive/My Drive/ip_hw_3


In [0]:
import os
import copy
import zipfile
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

import cv2

from PIL import Image

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

PATH_DATA = 'tl-signs-hse-itmo-2020-winter.zip'

In [5]:
x_all, y_all = [], []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    with zip_file.open('train.csv') as file:
        train_data = pd.read_csv(file)

    for _, x in tqdm(train_data.iterrows()):
        name, lable = x['filename'], x['class_number']
        with zip_file.open(os.path.join('train', 'train', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        x_all += [image]
        y_all += [lable]

num_classes = max(y_all) + 1

x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=0.1)

In [0]:
def get_hog(win_size, blockSize, blockStride, cellSize, nbins):
    win_size = (win_size, win_size)
    blockSize = (blockSize, blockSize)
    blockStride = (blockStride, blockStride)
    cellSize = (cellSize, cellSize)
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    hog = cv2.HOGDescriptor(win_size, blockSize, blockStride, cellSize, nbins, 
                            derivAperture, winSigma, histogramNormType, 
                            L2HysThreshold, gammaCorrection, nlevels)
    return hog

In [0]:
def calc_hog_desc(hog, images):
    xs = []
    for image in images:
        x = np.array(image, dtype=np.uint8)
        x = hog.compute(x).reshape(-1)
        xs += [x]

    return xs

In [0]:
def train_svm(xs, ys, degree):
    clf = svm.SVC(decision_function_shape='ovr', kernel='poly', degree=degree)
    clf.fit(xs, ys)
    return clf

In [0]:
def calc_accuracy(clf, xs, ys):
    return (clf.predict(xs) == ys).sum() / len(ys)

In [0]:
def train(x_train, y_train, x_val, y_val, 
          win_size, blockSize, blockStride, cellSize, nbins, 
          degree):
    print(f'win_size = {win_size} blockSize = {blockSize} '
          f'blockStride = {blockStride} cellSize = {cellSize} nbins = {nbins}')
    print(f'degree = {degree}')

    try:
        hog = get_hog(win_size, blockSize, blockStride, cellSize, nbins)

        x_hog_train = calc_hog_desc(hog, x_train)
        x_hog_val = calc_hog_desc(hog, x_val)
    except:
        print('FAILED')
        return

    clf = train_svm(x_hog_train, y_train, degree)

    accuracy_train = calc_accuracy(clf, x_hog_train, y_train)
    accuracy_val = calc_accuracy(clf, x_hog_val, y_val)

    print(f'Train accuracy = {accuracy_train}')
    print(f'Val accuracy = {accuracy_val}')
    print()

In [0]:
train(x_train, y_train, x_val, y_val, 
      win_size=8, blockSize=4, blockStride=4, cellSize=2, nbins=9, 
      degree=3)

train(x_train, y_train, x_val, y_val, 
      win_size=16, blockSize=8, blockStride=8, cellSize=4, nbins=9, 
      degree=3)

train(x_train, y_train, x_val, y_val, 
      win_size=32, blockSize=16, blockStride=16, cellSize=8, nbins=9, 
      degree=3)

train(x_train, y_train, x_val, y_val, 
      win_size=16, blockSize=16, blockStride=8, cellSize=8, nbins=9, 
      degree=3)

win_size = 8 blockSize = 4 blockStride = 4 cellSize = 2 nbins = 9
degree = 3


In [0]:
hog = get_hog(win_size=16, blockSize=16, blockStride=8, cellSize=8, nbins=9)

x_hog_all = calc_hog_desc(hog, x_all)

clf = train_svm(x_hog_all, y_all, degree=3)

In [0]:
x_test = []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    image_names = [f[10:] for f in zip_file.namelist() if 'test' in f]

    for name in tqdm(image_names):
        with zip_file.open(os.path.join('test', 'test', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        x_test += [image]

x_hog_test = calc_hog_desc(hog, x_test)
y_test = clf.predict(x_hog_test)

predictions = {'filename': image_names, 'class_number': y_test}

predictions = pd.DataFrame(predictions)
predictions.to_csv('test_classic.csv', index=False)